Se importan los paquetes necesarios

In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
import sys
from src.cleaning_functions import *

Se importan los datos como pandas DataFrame

In [3]:
sharks = pd.read_csv("./data/attacks.csv",encoding = "ISO-8859-1")

Se exploran los datos

In [5]:
sharks.shape

(25723, 24)

In [6]:
sharks.sample()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
22515,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
sharks.isna().sum()

Case Number               17021
Date                      19421
Year                      19423
Type                      19425
Country                   19471
Area                      19876
Location                  19961
Activity                  19965
Name                      19631
Sex                       19986
Age                       22252
Injury                    19449
Fatal (Y/N)               19960
Time                      22775
Species                   22259
Investigator or Source    19438
pdf                       19421
href formula              19422
href                      19421
Case Number.1             19421
Case Number.2             19421
original order            19414
Unnamed: 22               25722
Unnamed: 23               25721
dtype: int64

Se comprueba que es un DataFrame muy sucio con una gran cantidad de nulos

Se cambia el nombre de todas las columnas del DataFrame para simplificar

In [8]:
nuevas_columna = {columna:(columna.replace(" ", "_").replace(".", "_").lower() if columna[-1]!=" "
                  else columna.replace(" ", "_").replace(".", "_").lower()[:-1]) for columna in list(sharks.keys())}
sharks.rename(columns=nuevas_columna, inplace=True)

In [9]:
sharks.sample()

,case_number,date,year,type,country,area,location,activity,name,sex,...,species,investigator_or_source,pdf,href_formula,href,case_number_1,case_number_2,original_order,unnamed:_22,unnamed:_23
20359,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Ahora se va a proceder a la limpieza de los datos.

Lo primero es eliminar todas las filas con datos repetidos

In [10]:
sharks.drop_duplicates(inplace=True)

A continuación se van a eliminar todas aquellas filas donde todas las celdas sean nulos y todas las columnas donde todos los valores sean nulos

In [11]:
sharks.dropna(axis=0, how='all', inplace=True)
sharks.dropna(axis=1, how='all', inplace=True)

In [12]:
sharks.shape

(6311, 24)

Con este proceso se han limpiado bastantes filas.

In [13]:
sharks[(sharks["case_number"]=="0")].isna().sum()

case_number               0
date                      8
year                      8
type                      8
country                   8
area                      8
location                  8
activity                  8
name                      8
sex                       8
age                       8
injury                    8
fatal_(y/n)               8
time                      8
species                   8
investigator_or_source    8
pdf                       8
href_formula              8
href                      8
case_number_1             8
case_number_2             8
original_order            1
unnamed:_22               8
unnamed:_23               8
dtype: int64

Se comprueba que en aquellas columnas donde "case_number" es nulo, todo el resto de columnas salvo "original__order" son nulas también, por lo que no aportan información útil y se van a eliminar.

In [14]:
sharks.drop(sharks[(sharks["case_number"]=="0")].index,inplace= True)

Viendo la información del DF se ve que en las columnas "unnamed:_22" y "unnamed:_23" casi todos los valores son NaN. Se va a verificar cuales son aquellos valores no nulos y posteriormente eliminar las columnas completas

In [15]:
sharks[sharks["unnamed:_22"].notna()].shape

(1, 24)

In [16]:
sharks[sharks["unnamed:_22"].notna()].head()

,case_number,date,year,type,country,area,location,activity,name,sex,...,species,investigator_or_source,pdf,href_formula,href,case_number_1,case_number_2,original_order,unnamed:_22,unnamed:_23
1478,2006.05.27,27-May-2006,2006.0,Unprovoked,USA,Hawaii,"North Shore, O'ahu",Surfing,Bret Desmond,M,...,NaN,R. Collier,2006.05.27-Desmond.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2006.05.27,2006.05.27,4825.0,stopped here,NaN


In [17]:
sharks[sharks["unnamed:_23"].notna()].shape

(2, 24)

In [18]:
sharks[sharks["unnamed:_23"].notna()].head(2)

,case_number,date,year,type,country,area,location,activity,name,sex,...,species,investigator_or_source,pdf,href_formula,href,case_number_1,case_number_2,original_order,unnamed:_22,unnamed:_23
4415,1952.03.30,30-Mar-1952,1952.0,Unprovoked,NETHERLANDS ANTILLES,Curacao,NaN,Went to aid of child being menaced by the shark,A.J. Eggink,M,...,"Bull shark, 2.7 m [9'] was captured & dragged ...","J. Randall, p.352 in Sharks & Survival; H.D. B...",1952.03.30-Eggink.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1952.03.30,1952.03.30,1888.0,NaN,Teramo
5840,1878.09.14.R,Reported 14-Sep-1878,1878.0,Provoked,USA,Connecticut,"Branford, New Haven County",Fishing,Captain Pattison,M,...,NaN,"St. Joseph Herald, 9/14/1878",1878.09.14.R-Pattison.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1878.09.14.R,1878.09.14.R,463.0,NaN,change filename


No hay ningún dato valioso en esas columnas por lo que se van a eliminar

In [19]:
sharks.drop(columns = ["unnamed:_22", "unnamed:_23"],axis=1,inplace= True)

Ahora se va a estudiar la diferencia entre "case_number"case_number_1" y "case_number_2"

In [20]:
# Filas donde las columnas contienen diferente información
sharks[(sharks["case_number"]!=sharks["case_number_1"])|(sharks["case_number"]!=sharks["case_number_2"])|(sharks["case_number_1"]!=sharks["case_number_2"])][["case_number","case_number_1","case_number_2"]]

,case_number,case_number_1,case_number_2
34,2018.04.03,2018.04.02,2018.04.03
117,2017.07.20.a,2017/07.20.a,2017.07.20.a
144,2017.05.06,2017.06.06,2017.05.06
217,2016.09.15,2016.09.16,2016.09.15
314,2016.01.24.b,2015.01.24.b,2016.01.24.b
334,2015.12.23,2015.11.07,2015.12.23
339,2015.10.28.a,2015.10.28,2015.10.28.a
390,2015.07-10,2015.07.10,2015.07.10
560,2014.05.04,2013.05.04,2014.05.04
3522,1967.07.05,1967/07.05,1967.07.05


Se comprueba como las columnas "case_number", "case_number_1" y "case_number_2" contienen la misma información. En las celdas donde estas 3 columnas no coinciden la columna "case_number_1" es la que es diferente, y hay una pequeña cantidad de celdas donde "case_number" y "case_number_2" no coinciden y en ellas es la columna "case_number_2" la correcta. Debido a esto se van a eliminar "case_number" y "case_number_1" y renombrar "case_number_2"

In [21]:
sharks["date2"]=sharks["case_number_2"]
sharks.drop(columns = ["case_number","case_number_1","case_number_2"],axis=1,inplace= True)


In [22]:
sharks.sample(1)

,date,year,type,country,area,location,activity,name,sex,age,injury,fatal_(y/n),time,species,investigator_or_source,pdf,href_formula,href,original_order,date2
454,23-Jan-2015,2015.0,Unprovoked,SOUTH AFRICA,Eastern Cape Province,Nahoon Beach,Floating,Christiaan Nel,M,24,Foot bitten,N,Afternoon,NaN,"Daily Dispatch, 1/31/2015",2015.01.23-Nel.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,5849.0,2015.01.23


Hasta aquí se han limpiado del DataFrame todas aquellas filas con todos los valores NaN y todas aquellas columnas que no aportan nada de información útil, a continuación se va a exportar el DataFrame modificado a un .csv

In [23]:
sharks.to_csv("./data/sharks_limpio_raw.csv",index=False)

In [24]:
sharks = pd.read_csv("./data/sharks_limpio_raw.csv",encoding = "ISO-8859-1")

Se van a eliminar todas aquellas columnas que no se van a usar para sacar conclusiones

In [25]:
sharks.drop(columns = ["investigator_or_source","pdf","href_formula","href",
                       "original_order","name","area","location"],axis=1,inplace= True)

Ahora se va a hacer una limpieza de cada columna útil, para ello se van a utilizar las funciones del archivo de funciones. Se va a empezar por las columnas que contienen la información de la fecha

In [26]:
# Se limpia la columna del año
sharks["year"] = sharks["year"].apply(limpieza_year)
# Se limpia la columna de los meses
sharks["months_code"] = sharks["date"].apply(limpieza_meses)
# Se limpia la columna de los días
sharks["day"] = sharks["date"].apply(limpieza_dias)
# Se genera una columna con la fecha juntando el año mes y día
sharks["date1"] = sharks.year+"/"+sharks.months_code+"/"+sharks.day
sharks['date1'] = pd.to_datetime(sharks['date1'], format="%Y/%m/%d")
#Se eliminan las columnas de año, mes y día
sharks.drop(columns=["year","months_code","day"],inplace=True)

In [27]:
# Se limpia la columna "date2", que salió de la columna "case_number_2", que contiene información sobre la fecha

In [28]:
def limpieza_date2(x):
    if x != x:
        return np.nan
    x=str(x).replace(".", "-").replace(",","-")
    if x.split("-")[0].isdigit() and int(x.split("-")[0]) < 1700:
        return np.nan
    y=""
    for e in x:
        if e.isdigit() or e=="-":
            y+=e
    return y.strip("-")

sharks["date2"]=sharks["date2"].apply(limpieza_date2)

from datetime import datetime, timedelta

def todatetime2(x):
    if x != x:
        return np.nan
    try: return datetime.strptime(x,"%Y-%m-%d")
    except: return np.nan

sharks['date2'] = sharks['date2'].apply(todatetime2)
sharks['date2'] = sharks['date2'].apply(lambda x: str(x).replace(" 00:00:00",""))
sharks['date2'] = pd.to_datetime(sharks['date2'], format="%Y/%m/%d")

In [29]:
def todatetime(x):
    if x != x:
        return np.nan
    try: return datetime.strptime(x,"%d-%b-%Y")
    except: return np.nan
    
sharks['date'] = sharks['date'].apply(todatetime)


In [30]:
sharks[["date","date1","date2"]].head()

,date,date1,date2
0,2018-06-25,2018-06-25,2018-06-25
1,2018-06-18,2018-06-18,2018-06-18
2,2018-06-09,2018-06-09,2018-06-09
3,2018-06-08,2018-06-08,2018-06-08
4,2018-06-04,2018-06-04,2018-06-04


In [31]:
# Se van a estudiar las diferencias entre las 3 columnas de fecha del dataframe, para quedarse finalmente solo con 1
# Se van a visualizar las 3 columnas de date en aquellas filas donde al menos una de las 3 columnas sea no nula y haya diferencias entre las no nulas
sharks[
    ((sharks["date"]!=sharks["date1"]) | (sharks["date"]!=sharks["date2"]) | (sharks["date1"]!=sharks["date2"])) & 
    (sharks["date"].notna() | sharks["date1"].notna() | sharks["date2"].notna())] [["date","date1","date2"]]

,date,date1,date2
17,NaT,2018-04-30,2018-04-30
31,NaT,2018-04-10,2018-04-10
41,2018-02-18,2018-02-18,2018-02-17
59,NaT,2017-11-25,2017-11-25
62,NaT,2017-11-13,2017-11-13
...,...,...,...
6138,NaT,1786-08-10,1786-08-10
6139,NaT,1785-09-26,1785-09-26
6146,NaT,1771-07-12,1771-07-12
6150,NaT,1753-10-27,1753-10-27


In [32]:
# En aquellas filas donde la columna date sea nula y las otras dos sean iguales y no nulas, se rellena el nulo con el mismo valor que las otras dos columnas
for i in sharks[(sharks.date.isna()) & (sharks.date1 == sharks.date2) & (sharks.date1.notna())][["date","date1","date2"]].index:
    sharks.loc[i,"date"] = sharks.loc[i,"date1"]

# En aquellas filas donde la columna date2 sea nula y las otras dos sean iguales y no nulas, se rellena el nulo con el mismo valor que las otras dos columnas
for i in sharks[(sharks.date.notna()) & (sharks.date == sharks.date1) & (sharks.date2.isna())][["date","date1","date2"]].index:
    sharks.loc[i,"date2"] = sharks.loc[i,"date1"]

# En aquellas filas donde solo una de las 3 columnas sea no nula, se rellenan los otros dos nulos con el único valor no nulo
for i in sharks[(sharks.date.isna()) & (sharks.date1.isna()) & (sharks.date2.notna())][["date","date1","date2"]].index:
    sharks.loc[i,"date"] = sharks.loc[i,"date2"]
    sharks.loc[i,"date1"] = sharks.loc[i,"date2"]

for i in sharks[(sharks.date.isna()) & (sharks.date1.notna()) & (sharks.date2.isna())][["date","date1","date2"]].index:
    sharks.loc[i,"date"] = sharks.loc[i,"date1"]
    sharks.loc[i,"date2"] = sharks.loc[i,"date1"]

for i in sharks[(sharks.date.isna()) & (sharks.date1.notna()) & (sharks.date2.isna())][["date","date1","date2"]].index:
    sharks.loc[i,"date"] = sharks.loc[i,"date1"]
    sharks.loc[i,"date2"] = sharks.loc[i,"date1"]

# En aquellas filas donde dos columnas tienen el mismo valor, la tercera columna se iguala con las otras dos
for i in sharks[
    ((sharks["date"]!=sharks["date1"]) | (sharks["date"]!=sharks["date2"]) | (sharks["date1"]!=sharks["date2"])) & 
    (sharks["date"].notna() & sharks["date1"].notna() & sharks["date2"].notna())] [["date","date1","date2"]].index:
    
    if sharks.loc[i,"date"] == sharks.loc[i,"date1"]:
        sharks.loc[i,"date2"] = sharks.loc[i,"date1"]

    if sharks.loc[i,"date"] == sharks.loc[i,"date2"]:
        sharks.loc[i,"date1"] = sharks.loc[i,"date2"]

    if sharks.loc[i,"date1"] == sharks.loc[i,"date2"]:
        sharks.loc[i,"date"] = sharks.loc[i,"date1"]

# Cuando la columna "date1" y la "date2" tienen informaciones diferentes se van a eliminar ambos
for i in sharks[(sharks.date.isna()) & (sharks.date1.notna()) & (sharks.date2.notna()) & (sharks.date1!=sharks.date2)][["date","date1","date2"]].index:
    sharks.loc[i,"date1"] = np.nan
    sharks.loc[i,"date2"] = np.nan



In [33]:
# Se comprueba como ahora las tres columnas de fecha contienen la misma información 
pd.options.display.max_rows = 999
sharks[
    ((sharks["date"]!=sharks["date1"]) | (sharks["date"]!=sharks["date2"]) | (sharks["date1"]!=sharks["date2"])) & 
    (sharks["date"].notna() | sharks["date1"].notna() | sharks["date2"].notna())] [["date","date1","date2"]]

,date,date1,date2


In [34]:
# Se van a eliminar dos de las 3 columnas de fecha
sharks.drop(columns=["date1","date2"],inplace=True)

In [35]:
sharks["sex"] = sharks["sex"].apply(limpieza_sex)
sharks["age"] = sharks["age"].apply(limpieza_age)
sharks["fatal_(y/n)"] = sharks["fatal_(y/n)"].apply(limpieza_fatal)
sharks["hour"] = sharks["time"].apply(limpieza_time)
sharks["hour"] = sharks.hour.apply(lambda x: np.nan if x!=x else ("0"+str(x)+":00:00" if len(str(x))<2 else str(x)+":00:00"))
sharks.drop(columns = ["time"],axis=1,inplace= True)
sharks["type"] = sharks["type"].apply(limpieza_type)
sharks["species"] = sharks["species"].apply(limpieza_species)
sharks["injury"] = sharks["injury"].apply(limpieza_injury)
sharks["country"] = sharks["country"].apply(limpieza_country)
sharks["activity"] = sharks["activity"].apply(limpieza_activity)

Se reordena el DataFrame

In [36]:
sharks.sample()

,date,type,country,activity,sex,age,injury,fatal_(y/n),species,hour
1519,NaT,Boating,Australia,fishing,NaN,NaN,no injury,NO,white shark,15:00:00


In [37]:
sharks = sharks.iloc[:,[0,2,4,5,3,8,1,6,7]]
sharks.sample()

,date,country,sex,age,activity,species,type,injury,fatal_(y/n)
424,2015-05-15,United States of America,M,30.0,NaN,NaN,Provoked,injury,NO


Se eliminan las filas duplicadas y vacias

In [38]:
sharks.drop_duplicates(inplace=True)
sharks.dropna(axis=0, how='all', inplace=True)
sharks.dropna(axis=1, how='all', inplace=True)

In [39]:
sharks.shape

(6039, 9)

Se exporta el .csv limpio

In [40]:
sharks.to_csv("./data/shark_attacks.csv",index=False)

In [41]:
sharks.head()

,date,country,sex,age,activity,species,type,injury,fatal_(y/n)
0,2018-06-25,United States of America,F,57.0,NaN,white shark,Boating,no injury,NO
1,2018-06-18,United States of America,F,11.0,standing,NaN,Provoked,injury,NO
2,2018-06-09,United States of America,M,48.0,surf,NaN,NaN,injury,NO
3,2018-06-08,Australia,M,NaN,surf,NaN,Provoked,injury,NO
4,2018-06-04,Mexico,M,NaN,diving,tiger shark,Provoked,injury,NO


In [42]:
sharks["fatal_(y/n)"].value_counts()

NO     4139
YES    1304
Name: fatal_(y/n), dtype: int64

In [43]:
sharks["injury"].value_counts()

injury       3951
fatal        1272
no injury     788
Name: injury, dtype: int64

In [44]:
sharks[(sharks["injury"]!= "fatal") & (sharks["fatal_(y/n)"]=="YES") & (sharks["injury"].notna())].injury.value_counts()

injury       82
no injury     1
Name: injury, dtype: int64

In [45]:
sharks[(sharks["injury"]== "fatal") & (sharks["fatal_(y/n)"]=="NO")]

,date,country,sex,age,activity,species,type,injury,fatal_(y/n)
6182,NaT,Kenya,M,NaN,diving,NaN,Provoked,fatal,NO
